In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

In [4]:
df.head()

question      answer
0                   What is the capital of France?       Paris
1                  What is the capital of Germany?      Berlin
2               Who wrote 'To Kill a Mockingbird'?  Harper-Lee
3  What is the largest planet in our solar system?     Jupiter
4   What is the boiling point of water in Celsius?         100

In [5]:
# tokeize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','').replace("'","")
  return text.split()

In [6]:
tokenize("yo whassup mate's hows doing?")

['yo', 'whassup', 'mates', 'hows', 'doing']

In [7]:
# vocab
vocab = {'<UNK>':0}

def buil_vocab(row):
  tokenize_question = tokenize(row['question'])
  tokenize_answer = tokenize(row['answer'])
  merged_token = tokenize_question + tokenize_answer

  for token in merged_token:
    if token not in vocab:
      vocab[token] = len(vocab)

In [8]:
df.apply(buil_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [9]:
len(vocab)

324

In [10]:
# convert words to numerical indices
def convert_to_indices(text, vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

In [12]:
class QADataset(Dataset):
  def __init__(self,df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    num_ques = convert_to_indices(self.df.iloc[idx]['question'],self.vocab)
    num_ans = convert_to_indices(self.df.iloc[idx]['answer'], self.vocab)
    return torch.tensor(num_ques), torch.tensor(num_ans)

In [13]:
dataset = QADataset(df, vocab)

In [14]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [15]:
for question, answer in dataloader:
  print(question, answer)


tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[ 1,  2,  3, 92, 93, 94]]) tensor([[95]])
tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]]) tensor([[199]])
tensor([[ 42, 137,   2, 138,  39, 175, 269]]) tensor([[99]])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([[260]])
tensor([[ 42, 255,   2, 256,  83, 257, 258]]) tensor([[259]])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([[121]])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([[136]])
tensor([[10, 75, 76]]) tensor([[77]])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([[68]])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([[238]])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([[173]])
tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([[188]])
tensor([[ 1,  2,  3,  4,  5, 99]]) tensor([[100]])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([[225]])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([[233]])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([[287]])
tensor([[ 1,  2,  3, 5

In [16]:
import torch.nn as nn

In [29]:
class simpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(input_size=50, hidden_size=64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_ques = self.embedding(question)
    hidden, final = self.rnn(embedded_ques)
    output = self.fc(final.squeeze(0))
    return output

In [18]:
# Explanation of why use batch_first = True and squeeze
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(input_size=50, hidden_size=64)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)

b=x(a)
print("shape of b:", b.shape)

c,d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

d = z(d)
print("shape of d:", d.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 6, 324])


RNN give output 2 values 1. hidden list 2. Final list

1. In final list we want [1,1,64] but RNN (pytorch) gives [1,6,64]. Thats why I use `batch_first = True`.
2. Final output dim [1,324] for finding loss but our final output dim is [1,1,324]. So we use `self.fc(final.squeez(0)) `

In [26]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(input_size=50, hidden_size=64, batch_first = True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)

b=x(a)
print("shape of b:", b.shape)

c,d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

d = z(d.squeeze(0))
print("shape of d:", d.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of d: torch.Size([1, 324])


In [30]:
learning_rate = 0.001
epochs = 20

In [31]:
model = simpleRNN(len(vocab))

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
# training loop
for epoch in range(epochs):
  total_loss = 0
  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss
    loss = criterion(output, answer[0])

    # gradient
    loss.backward()

    # update
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch {epoch+1}, Loss: {total_loss}")

Epoch 1, Loss: 529.304425239563
Epoch 2, Loss: 454.9531955718994
Epoch 3, Loss: 374.72557497024536
Epoch 4, Loss: 314.9335927963257
Epoch 5, Loss: 264.4515221118927
Epoch 6, Loss: 215.62289237976074
Epoch 7, Loss: 172.60767030715942
Epoch 8, Loss: 135.38213694095612
Epoch 9, Loss: 104.33204558491707
Epoch 10, Loss: 80.1642056107521
Epoch 11, Loss: 61.48604401946068
Epoch 12, Loss: 47.98003838956356
Epoch 13, Loss: 38.18657709658146
Epoch 14, Loss: 30.475939884781837
Epoch 15, Loss: 25.038906790316105
Epoch 16, Loss: 20.623703606426716
Epoch 17, Loss: 17.29247435927391
Epoch 18, Loss: 14.599822282791138
Epoch 19, Loss: 12.59281226247549
Epoch 20, Loss: 10.82304223626852


In [50]:
def predict(model, question, threshold =0.5):
  # convert question to number
  num_ques = convert_to_indices(question, vocab)

  # tensor
  ques_tensor = torch.tensor(num_ques).unsqueeze(0)

  # send to model
  output = model(ques_tensor)

  # convert logit to prob
  probs = torch.nn.functional.softmax(output, dim=1)

  value, index = torch.max(probs, dim=1)
  print( value, index)

  if value < threshold:
    print("I don't know the answer")
  else:
    print(list(vocab.keys())[index])

In [51]:
predict(model, "what is the largest planet in our solar system")

tensor([0.9108], grad_fn=<MaxBackward0>) tensor([23])
jupiter
